In [ ]:
import requests
import json
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)

In [ ]:
# fetch api key from file

with open('secret/key.txt', 'r') as file:
    api_key = file.read()
#api_key

'c801ef08a40fac752083082acadfefe13ac5bba9'

In [64]:
#keywords = ["swollen ankles", "horny goat weed", "scalp psoriasis"]

url = "https://google.serper.dev/search"
location = "Calgary,Alberta,Canada"
language = "en"
device = "desktop"

In [229]:
# peforms a serch on one kw and returns the Xth page of relults

def search_kw(kw, page=1, url="https://google.serper.dev/search"):
    payload = json.dumps({
      "q": kw, # keyword to search for
      "gl": "us", # location
      "hl": "en", # language
      "autocorrect": False,
      "page": page
    })

    headers = {
      "X-API-KEY": api_key,
      "Content-Type": "application/json"
    }

    response = requests.request("POST", url=url, headers=headers, data=payload)
    return(response)

In [320]:
# runs search_ky over multiple pages and returns them in a list

def get_pages(page_count, kw, url="https://google.serper.dev/search"):
    pages = list()
    for page in range(1, page_count+1):
        pages.append(search_kw(kw, page, url))
    return pages

In [302]:
# extract kw, page, rank, and link from 1 page of response info and return df

def extract_response(response):
    df = pd.DataFrame(columns = ['kw', 'page', 'rank','link'])
    q = response.json()['searchParameters']['q']
    page = response.json()['searchParameters']['page']
    for rank, result in enumerate(response.json()["organic"], 1):
        link = result['link']
        df.loc[len(df)] = [q, page, rank, link]
    return(df)

In [347]:
# takes a list of pages of response, iterates extract_response(), extracts and renumbers responses, returns df of ky, rank, link

def collate_pages(pages, max_rank=None):
    df = pd.DataFrame()
    for response in pages:
        data = extract_response(response)
        df = pd.concat([df, data], ignore_index=True)
    df = df.reset_index(drop=True)
    df["rank"] = df.index + 1
    if max_rank is not None:
        df = df[df['rank'] <= max_rank]
    df.drop(["page"], axis=1, inplace=True)
    return df

In [370]:
# accepty a df containing "Keywords" and iterates collate_pages() returns df of kw

def process_kws(df_in):
    df_out = pd.DataFrame()
    keywords = df_in["Keyword"]
    for kw in keywords:
        kw_pages = get_pages(3, kw) # get 3 pages (<= 30 records) for each kw
        df = collate_pages(kw_pages, 20) # colate pages, limit to top 20 records.
        df_out = pd.concat([df_out, df], ignore_index=True)
    return(df_out)

In [378]:
# load kw
df_in = pd.read_csv('data/kw_1_000.csv')
#df_in = df_in.head() # FOR TESTING PRUPOSES ONLY BECAUSE I AM LIMITED BY MY QUERY QUOTA OF 1440
df_in

,Keyword,SERP features,Volume,URL
0,side table lamps,"Image pack, Thumbnail, Shopping results, Sitel...",2100,https://www.thespruce.com/how-to-pick-a-bedsid...
1,bell's palsy causes,"Featured snippet, People also ask, Sitelinks, ...",12000,https://www.webmd.com/brain/understanding-bell...
2,snap in dentures,"Sitelinks, People also ask, Video preview",7600,https://www.webmd.com/oral-health/what-to-know...
3,hoodies for men,"Shopping results, Thumbnail, People also ask",38000,https://www.amazon.com/Mens-Fashion-Hoodies-Sw...
4,blushing philodendron,"People also ask, Sitelinks, Knowledge panel, S...",2300,https://www.thespruce.com/grow-phildendron-eru...
...,...,...,...,...
995,paint colors that go with beige walls,"Featured snippet, People also ask, Sitelinks, ...",1000,https://www.thespruce.com/colors-that-go-with-...
996,prime days,"Sitelinks, People also ask",15000,https://www.amazon.com/primeday
997,what is fingering,"Featured snippet, Sitelinks",7100,https://www.webmd.com/sex/what-is-fingering
998,zinus bed frame,"Adwords top, Adwords sitelink, Sitelinks, Shop...",16000,https://www.amazon.com/Beds-Frames-Bases-Zinus...


In [380]:
df_400 = df_in.iloc[0:400]

In [381]:
df = process_kws(df_400)

In [382]:
df.to_csv('data/first_400.csv')

In [383]:
df

,kw,rank,link
0,side table lamps,1,https://www.target.com/c/table-lamps-lighting-...
1,side table lamps,2,https://www.amazon.com/Side-Table-Lamps/s?k=Si...
2,side table lamps,3,https://www.wayfair.com/lighting/sb1/bedside-t...
3,side table lamps,4,https://www.ikea.com/us/en/cat/table-lamps-10732/
4,side table lamps,5,https://www.crateandbarrel.com/lighting/table-...
...,...,...,...
7985,raisins nutrition,16,https://www.checkyourfood.com/ingredients/ingr...
7986,raisins nutrition,17,https://healthyeating.sfgate.com/health-benefi...
7987,raisins nutrition,18,https://www.wellandgood.com/benefits-of-raisins/
7988,raisins nutrition,19,https://ratinkhosh.com/disadvantages-of-eating...


In [236]:
# print response
def print_response(response):
    print (f"Results for keyword: \"{response.json()['searchParameters']['q']}\"")
    for rank, result in enumerate(response.json()["organic"], 1):
        print(f"Rank: {rank}, Title: {result['title']}, Link: {result['link']}")

In [208]:
# OLD DEPRICATED

def search_kw_list(keywords, page=1, url="https://google.serper.dev/search"):
    results_list = []
    for keyword in keywords:
        payload = json.dumps({
            "q": keyword,
            "gl": "us",
            "hl": "en",
            "autocorrect": False,
            "page": page
        })
        headers = {
            'X-API-KEY': api_key,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        if response.status_code == 200:
            results_list.append(response.json())
        else:
            print(f"Error: Failed to retrieve search results for {keyword}. Status code {response.status_code}")
    return results_list

In [209]:
results_list = search_kw_list(keywords, num_results)

In [211]:
# OLD DEPRICTED

def print_results(results_list):
    for results in results_list:
        print(f"Results for keyword: {results['searchParameters']['q']}")
        for result in results["organic"]:
            print(f"Rank: {result['position']}\nTitle: {result['title']}\nLink: {result['link']}\n")

In [354]:
# Print the title and link of each organic search result for each keyword
#for results in results_list:
#    for result in results["organic"]:
#        print(f"Rank: {result['position']}\nTitle: {result['title']}\nLink: {result['link']}\n")

In [356]:
# OLD DEPRICATED?????????

def search_kw_list():
    for keyword in keywords:
        url = "https://google.serper.dev/search"
        payload = json.dumps({
            "q": keyword,
            "gl": "us",
            "hl": "en",
            "num": 10,
            #"autocorrect": True
        })
        headers = {
            'X-API-KEY': api_key,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        print(f"Keyword: {keyword}")
        if response.status_code == 200:
            search_results = response.json()
            for result in search_results["organic"]:
                print(f"Rank: {result['position']}\nTitle: {result['title']}\nLink: {result['link']}\n")
        else:
            print(f"Error: Failed to retrieve search results for {keyword}.")

In [27]:
print(f"Rank: {result['position']}\nTitle: {result['title']}\nLink: {result['link']}\n")

Rank: 6
Title: Scalp psoriasis vs. seborrheic dermatitis: What's the difference? - Mayo Clinic
Link: https://www.mayoclinic.org/diseases-conditions/psoriasis/expert-answers/scalp-psoriasis/faq-20058544

